In [1]:
import tensorflow as tf
from mlxtend.data import loadlocal_mnist
import numpy as np
import keras
tf.logging.set_verbosity(tf.logging.INFO)




Using TensorFlow backend.


In [2]:
((train_data, train_labels),
 (test_data, test_labels)) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
target_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot',
}


In [4]:
train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)

test_data = test_data/np.float32(255)
test_labels = test_labels.astype(np.int32)

In [5]:
classes = 10

# Applying dropout

In [36]:
def cnn_model_fn(features,labels,mode): 
    #INput layer
    input_layer=tf.reshape(features["x"],[-1,28,28,1])
    #Convolution layer 1
    conv1=tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
    #POoling layer 1
    pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    #Convolution layer 2
    conv2=tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
    #POoling layer 2
    pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    conv3=tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
    #POoling layer 1
    pool3=tf.layers.max_pooling2d(inputs=conv3,pool_size=[2,2],strides=2)
    
    #DEnse layer
    pool3_flat=tf.reshape(pool3,[-1,3*3*128])
    dense=tf.layers.dense(inputs=pool3_flat,units=1024,activation=tf.nn.relu)
    dropout=tf.layers.dropout(inputs=dense,rate=0.2,training=mode==tf.estimator.ModeKeys.TRAIN)
    #Logits Layer
    logits=tf.layers.dense(inputs=dropout,units=10)
    predictions={"classes":tf.argmax(input=logits,axis=1),"probabilities":tf.nn.softmax(logits,name="softmax_tensor")}
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits, scope = 'loss')
    # Configure the Training Op (for TRAIN mode)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    

In [37]:
#X_train,Y_train=loadlocal_mnist(images_path='train-images-idx3-ubyte',labels_path='train-labels-idx1-ubyte')
#X_test,Y_test=loadlocal_mnist(images_path='t10k-images-idx3-ubyte',labels_path='t10k-labels-idx1-ubyte')

In [38]:
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

In [39]:
#Y_train = np.eye(10)[Y_train.astype('int32')]
#Y_test = np.eye(10)[Y_test.astype('int32')]

In [40]:
#Y_train.shape

In [41]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpght3_tmz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9cdee1ad30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
"""# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)"""

'# Set up logging for predictions\ntensors_to_log = {"probabilities": "softmax_tensor"}\n\nlogging_hook = tf.train.LoggingTensorHook(\n    tensors=tensors_to_log, every_n_iter=50)'

In [43]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=128,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(input_fn=train_input_fn, steps=1500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpght3_tmz/model.ckpt.
INFO:tensorflow:loss = 2.3013024, step = 1
INFO:tensorflow:global_step/sec: 9.99334
INFO:tensorflow:loss = 2.144712, step = 101 (10.008 sec)
INFO:tensorflow:global_step/sec: 9.46427
INFO:tensorflow:loss = 1.2981548, step = 201 (10.566 sec)
INFO:tensorflow:global_step/sec: 9.28211
INFO:tensorflow:loss = 0.98828447, step = 301 (10.773 sec)
INFO:tensorflow:global_step/sec: 9.28535
INFO:tensorflow:loss = 0.78056836, step = 401 (10.770 sec)
INFO:tensorflow:global_step/sec: 9.27967
INFO:tensorflow:loss = 0.65421057, step = 501 (10.776 sec)
INFO:tensorflow:global_step/sec: 9.26917
INFO:tensorflow:loss = 0.7914592, step = 601 (10.788 sec)
INFO:tensorflow:global_step/sec: 9.01213
INFO:ten

In [44]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-19T09:08:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpght3_tmz/model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-19-09:08:49
INFO:tensorflow:Saving dict for global step 1500: accuracy = 0.7967, global_step = 1500, loss = 0.56943244
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1500: /tmp/tmpght3_tmz/model.ckpt-1500
{'accuracy': 0.7967, 'loss': 0.56943244, 'global_step': 1500}


# Batch Normalisation

In [52]:
def cnn_model_fn_batchnormalisation(features,labels,mode): 
    #INput layer
    input_layer=tf.reshape(features["x"],[-1,28,28,1])
    #Convolution layer 1
    conv1=tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding="same")
    conv1=tf.layers.batch_normalization(conv1)
    conv1=tf.nn.relu(conv1)
    
    #POoling layer 1
    pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    #Convolution layer 2
    conv2=tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],padding="same")
    conv2=tf.layers.batch_normalization(conv2)
    conv2=tf.nn.relu(conv2)
    
    #POoling layer 2
    pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    #DEnse layer
    #convoution layer 3
    conv3=tf.layers.conv2d(inputs=pool2,filters=128,kernel_size=[5,5],padding="same")
    conv3=tf.layers.batch_normalization(conv3)
    conv3=tf.nn.relu(conv3)
    
    #POoling layer 3
    pool3=tf.layers.max_pooling2d(inputs=conv3,pool_size=[2,2],strides=2)
    
    pool3_flat=tf.reshape(pool3,[-1,3*3*128])
    dense=tf.layers.dense(inputs=pool3_flat,units=1024,activation=tf.nn.relu)
    #dropout=tf.layers.dropout(inputs=dense,rate=0.4,training=mode==tf.estimator.ModeKeys.TRAIN)
    #Logits Layer
    logits=tf.layers.dense(inputs=dense,units=10)
    predictions={"classes":tf.argmax(input=logits,axis=1),"probabilities":tf.nn.softmax(logits,name="softmax_tensor")}
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits, scope = 'loss')
    # Configure the Training Op (for TRAIN mode)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    

In [53]:
# Create the Estimator
mnist_classifier1 = tf.estimator.Estimator(model_fn=cnn_model_fn_batchnormalisation)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4767yda5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9d03247a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [54]:
train_input_fn1 = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier1.train(input_fn=train_input_fn1, steps=1500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp4767yda5/model.ckpt.
INFO:tensorflow:loss = 2.3001745, step = 1
INFO:tensorflow:global_step/sec: 4.25522
INFO:tensorflow:loss = 2.1465077, step = 101 (23.502 sec)
INFO:tensorflow:global_step/sec: 4.34372
INFO:tensorflow:loss = 1.1962942, step = 201 (23.022 sec)
INFO:tensorflow:global_step/sec: 4.38753
INFO:tensorflow:loss = 0.8965979, step = 301 (22.792 sec)
INFO:tensorflow:global_step/sec: 4.49594
INFO:tensorflow:loss = 0.9095679, step = 401 (22.242 sec)
INFO:tensorflow:global_step/sec: 4.51634
INFO:tensorflow:loss = 0.8509109, step = 501 (22.142 sec)
INFO:tensorflow:global_step/sec: 4.37755
INFO:tensorflow:loss = 0.78899693, step = 601 (22.844 sec)
INFO:tensorflow:global_step/sec: 4.23131
INFO:tens

In [55]:
eval_input_fn1 = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier1.evaluate(input_fn=eval_input_fn1)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-19T09:38:10Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4767yda5/model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-19-09:38:15
INFO:tensorflow:Saving dict for global step 1500: accuracy = 0.771, global_step = 1500, loss = 0.6083943
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1500: /tmp/tmp4767yda5/model.ckpt-1500
{'accuracy': 0.771, 'loss': 0.6083943, 'global_step': 1500}
